In [1]:
# 1) Mean/Median Age (Age_Days) - Descriptive Statistics Mean/IQR
# 2) Mean/Median Weight (WT_KGS) - Descriptive Statistics Mean/IQR
# 3) Gender distribution (GENDER) - significant differences? - One Way Chi Squre
# 4) Most common Breeds (BREED) - One Way Chi Squre
# 5) Most common shunts (SHUNT) - One Way Chi Squre
# 6) Most common portal vein origin (PORTAL_V) - One Way Chi Squre
# 7) Most common systemic vein insertion (SYSTEM_V) - One Way Chi Squre
# 8) Most common contributing vessels (CONTRB_V) - One Way Chi Squre
# 9) PP score distribution (PP_Score) - One Way Chi Squre and Descriptive Statistics Mean/IQR (age, wt, and pps_n are set as continuous exploratory)

# Cross tab frequencies and ANOVA
# HOSPITAL COMPARISONS (HOSPITAL):
# 10) Do the above values differ AMONG HOSPITALS (HOSPITAL) - Two Way Chi Squre (Comprehensive pairwise comparisons reveals)

# Cross tab frequencies and ANOVA
# COUNTRY COMPARISONS (COUNTRY):
# 11) Do the above values differ AMONG COUNTRIES (COUNTRY)  - Two Way Chi Squre (Comprehensive pairwise comparisons reveals)

# Cross tab frequencies and ANOVA
# BREED COMPARISONS (BREED): [5 MOST COMMON BREEDS]
# 12) What is Gender distribution (GENDER) comparison among 5 most common breeds (BREED)? - Two Way Chi Squre (Comprehensive pairwise comparisons reveals)
# 13) What is Age distribution (Age_Days) comparison among 5 most common breeds (BREED)? - Sub ANOVA limited to breeds 61, 38, 53, 35, 47
# 14) What are 5 most common Shunts  (SHUNT) comparison among 5 most common breeds (BREED)? - Two Way Chi Squre (Comprehensive pairwise comparisons reveals)
# 15) What is PP Score (PP_Score) comparison among 5 most common breeds (BREED)? # Probably best looked at categorically with two-way
# comprehensive chi-square for contingency table first time (Comprehensive pairwise comparisons reveals)


# Regression and ANOVA
# AGE COMPARISONS (Age_Days):
# 16) Does AGE (Age_Days) correlate with Breed (BREED) - RQ 13 handles this with age as DV and Breed (Top 5) as IV with 5 class levels fo breed
# 17) Does AGE (Age_Days) correlate with PP Score (PP_Score) - Standard ANOVA but similar issue with shunt type having a lot of cats, see 18
# 18) Does AGE (Age_Days) correlate with Shunt (SHUNT) - There are 44 different shunt types (shunt type is not ordinal) and age is continuous. 
#  An ANOVA would 
# not yield meaningful results iwth the 44 goups giving a mean age for pairise comparisons.  We could contribute collapsing shunt and 
# bucketing age.


# SHUNT COMPARISONS (SHUNT) ANOVA:
# 19) Does Shunt (SHUNT) correlate with Weight (WT_KGS)    [Above and below mean/median?] - see 20
# 20) Does Shunt (SHUNT) correlate with PP Score (PP_Score) There are 44 different shunt types (non-ordinal) and 11 (with one missing) pp Score.  
# This could be a 
# either a huge cross tab or a difficlt it interpret ANOVA.   Need to define DV/IV an consider collpasing (top 5?) Shunt Type. See 17 and 18

# Breed ANOVA and Cross Tab
# These (21, 22, 23) are all caegorical with multiple and in most cases non-ordinal categorization making inelgible for regression and 
# multiple categories difficult to interpret with thin data classes for ANOVA.  Suggest first round (Comprehensive pairwise comparisons reveals)
# 21) Does PV origin (PORTAL_V) correlate with breed (BREED) # Cross Tab Frequency - Two Way Chi Square
# 22) Does systemic v insertion (SYSTEM_V) correlate with breed (BREED) # Cross Tab Frequency - Two Way Chi Square
# 23) Do contributing vessels (CONTRB_V) correlate with breed (BREED) # Cross Tab Frequency - Two Way Chi Square

In [1]:
from tabulate import tabulate
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages
from datetime import datetime
today = datetime.today().strftime("%Y-%m-%d")
now = datetime.now()


#Stats Import
from scipy.stats import wilcoxon 
# from numpy.random import seed
# from numpy.random import math

import pingouin as pg
from pingouin import pairwise_ttests, pairwise_tukey, pairwise_gameshowell
import statsmodels.stats.multicomp as mc
import scipy.stats as stats
import statsmodels.api as sm
from bioinfokit.analys import stat
from statsmodels.formula.api import ols

sys.path.append(os.path.join(sys.path[0],'C:\Junk\lamb_consulting'))
sys.path.append(os.path.join(sys.path[0],'U:\lamb_consulting'))


# import cox, means, regressions

from lamb_consulting import *

from lamb_consulting.means import means_desc, means_desc_class, aov
from lamb_consulting.freqs import freqs, chi_contingency, chi_oneway
from lamb_consulting.export import export_to_Excel
# from lamb_consulting.regressions import ols, regression_plot
from lamb_consulting.cox import cox
from lamb_consulting.kaplan_meier import km,km_plots


def freqme(ds,var1):
    f=ds[var1].value_counts(dropna=False)
    p=ds[var1].value_counts(normalize=True)
    df=pd.concat({f,p}, axis=1, keys=['frequency', 'percent'])
    df["cumfrequency"] = df["frequency"].cumsum()
    df["cumpercent"] = df["percent"].cumsum()
    return df

In [2]:
print(os.getcwd())
# tpath = "C:/Junk/Weisse/EHPSS/"
tpath = "U:/Fox/Weisse/EHPSS/"
fn="EHPSS20221207"

extension='.xlsx'
file = tpath+fn+extension
file
df = pd.read_excel((file), sheet_name='Data')

df['contrb_v'].fillna(method='ffill')

def male_n(row):
    if row["gender"] == 1 or row["gender"] == 2:
        return 1
    elif row["gender"] == 3 or row["gender"] == 4:
        return 0
    else:
        return -1


def male_c(row):
    if row["gender_1"] == 'MC' or row["gender_1"] == 'MI' or row["gender_1"] == 'NM' or row["gender_1"] == 'MN' or row["gender_1"] == 'ME' or row["gender_1"] == 'M':
        return "M"
    elif row["gender_1"] == 'FS' or row["gender_1"] == 'FI' or row["gender_1"] == 'FE': 
        return "F"
    else:
        return "error"


df['male_n'] = df.apply(male_n,axis=1)
df['male_c'] = df.apply(male_c,axis=1)

cont = ['age_d', 'wt_kg', 'pps_n']
cat = ['male_n', 'breed_n', 'shunt_ty', 'portal_v', 'system_v', 'contrb_v', 'hospital', 'country', 'pps_n']
# cat = ['male_n', 'breed_c']

cont_describe = "age_d wt_kg pps_n"
cat_describe = "male_c breed_nc shunt_ty portal_v system_v contrb_v hospital country pps_n"


# print(df["gender_1"].value_counts())
# print(df["gender"].value_counts())

# print(df["male_c"].value_counts())
# print(df["male_n"].value_counts())


u:\Fox\Weisse\EHPSS


# One Way Frequency

In [4]:

my_list = cat.copy()
my_list.remove("breed_n")
my_list += ['breed_n']

cat_one = pd.DataFrame()
for i in my_list:
    cat_one = cat_one.append(chi_oneway(df,i))


cat_one


,table,var_level,frequency,percent,cumfrequency,cumpercent,p_value
0,Table male_n,0.0,575,0.531423,575,0.531423,0.03871
1,Table male_n,1.0,507,0.468577,1082,1.000000,0.03871
0,Table shunt_ty,1.0,295,0.272643,295,0.272643,0.00000
1,Table shunt_ty,2.0,208,0.192237,503,0.464880,0.00000
2,Table shunt_ty,3.0,165,0.152495,668,0.617375,0.00000
...,...,...,...,...,...,...,...
59,Table breed_n,7.0,1,0.000924,1078,0.996303,0.00000
60,Table breed_n,65.0,1,0.000924,1079,0.997227,0.00000
61,Table breed_n,28.0,1,0.000924,1080,0.998152,0.00000
62,Table breed_n,10.0,1,0.000924,1081,0.999076,0.00000


# Create Top 5 Breed dataset based of of the one-way Chi-Squre Output

In [5]:
# breed_t5_list = [61, 38, 53, 35, 47]
breed_t5_list = df['breed_n'].value_counts().keys()[0:5].sort_values(ascending=False).tolist()
breed_mask = (df['breed_n'].isin(breed_t5_list)) 
breed_t5 = df.loc[breed_mask]
print(breed_t5_list)
print(df.shape)
print(breed_t5.shape)

[61.0, 53.0, 47.0, 38.0, 35.0]
(1082, 27)
(605, 27)


# Two Way Frequency

In [5]:
import itertools
iter_list = list(itertools.combinations(cat, 2))
iter_list

for i in iter_list:
    cross_tmp = chi_contingency(df,"hospital",i,"hospital","chi_cont")
    cross_cat = cross_cat.append(cross_tmp)

cross_tmp = chi_contingency(df,"country","pps_n","country","chi_cont")
cross_cat = cross_cat.append(cross_tmp)




[('male_n', 'breed_n'),
 ('male_n', 'shunt_ty'),
 ('male_n', 'portal_v'),
 ('male_n', 'system_v'),
 ('male_n', 'contrb_v'),
 ('male_n', 'hospital'),
 ('male_n', 'country'),
 ('male_n', 'pps_n'),
 ('breed_n', 'shunt_ty'),
 ('breed_n', 'portal_v'),
 ('breed_n', 'system_v'),
 ('breed_n', 'contrb_v'),
 ('breed_n', 'hospital'),
 ('breed_n', 'country'),
 ('breed_n', 'pps_n'),
 ('shunt_ty', 'portal_v'),
 ('shunt_ty', 'system_v'),
 ('shunt_ty', 'contrb_v'),
 ('shunt_ty', 'hospital'),
 ('shunt_ty', 'country'),
 ('shunt_ty', 'pps_n'),
 ('portal_v', 'system_v'),
 ('portal_v', 'contrb_v'),
 ('portal_v', 'hospital'),
 ('portal_v', 'country'),
 ('portal_v', 'pps_n'),
 ('system_v', 'contrb_v'),
 ('system_v', 'hospital'),
 ('system_v', 'country'),
 ('system_v', 'pps_n'),
 ('contrb_v', 'hospital'),
 ('contrb_v', 'country'),
 ('contrb_v', 'pps_n'),
 ('hospital', 'country'),
 ('hospital', 'pps_n'),
 ('country', 'pps_n')]

In [3]:
import itertools
iter_list = list(itertools.combinations(cat, 2))
iter_list


[('male_n', 'breed_n'),
 ('male_n', 'shunt_ty'),
 ('male_n', 'portal_v'),
 ('male_n', 'system_v'),
 ('male_n', 'contrb_v'),
 ('male_n', 'hospital'),
 ('male_n', 'country'),
 ('male_n', 'pps_n'),
 ('breed_n', 'shunt_ty'),
 ('breed_n', 'portal_v'),
 ('breed_n', 'system_v'),
 ('breed_n', 'contrb_v'),
 ('breed_n', 'hospital'),
 ('breed_n', 'country'),
 ('breed_n', 'pps_n'),
 ('shunt_ty', 'portal_v'),
 ('shunt_ty', 'system_v'),
 ('shunt_ty', 'contrb_v'),
 ('shunt_ty', 'hospital'),
 ('shunt_ty', 'country'),
 ('shunt_ty', 'pps_n'),
 ('portal_v', 'system_v'),
 ('portal_v', 'contrb_v'),
 ('portal_v', 'hospital'),
 ('portal_v', 'country'),
 ('portal_v', 'pps_n'),
 ('system_v', 'contrb_v'),
 ('system_v', 'hospital'),
 ('system_v', 'country'),
 ('system_v', 'pps_n'),
 ('contrb_v', 'hospital'),
 ('contrb_v', 'country'),
 ('contrb_v', 'pps_n'),
 ('hospital', 'country'),
 ('hospital', 'pps_n'),
 ('country', 'pps_n')]

In [19]:
print(len(iter_list))
cross_cat = pd.DataFrame()

my_len = (len(iter_list))
for i in range(0, len(iter_list)):
    # print(i)
    res = [iter_list[i]]
    # print(res)
    # print(type(res))
    a = res[0]
    # print(type(a))
    v1,v2 = a
    # print(v1)
    # print(v2)
    # type(x)
    cross_tmp = chi_contingency(df,v1,v2,"1","chi_cont")
    cross_cat = cross_cat.append(cross_tmp)

type(cross_cat)
cross_cat


36


,r_q_n,analysis,research_q,Table,Frequency,Percent,Missing,iv1,iv2
0,1,chi_cont,1,Table male_n * breed_n,2,0.187441,.,0,1.0
1,1,chi_cont,1,Table male_n * breed_n,0,0.000000,.,0,2.0
2,1,chi_cont,1,Table male_n * breed_n,0,0.000000,.,0,3.0
3,1,chi_cont,1,Table male_n * breed_n,2,0.187441,.,0,4.0
4,1,chi_cont,1,Table male_n * breed_n,4,0.374883,.,0,5.0
...,...,...,...,...,...,...,...,...,...
16,1,chi_cont,1,Table country * pps_n,8,0.742115,.,3,2.0
17,1,chi_cont,1,Table country * pps_n,16,1.484230,.,3,3.0
18,1,chi_cont,1,Table country * pps_n,30,2.782931,.,3,4.0
19,1,chi_cont,1,Table country * pps_n,83,7.699443,.,3,5.0


In [6]:

# def chi_contingency(ds,var1,var2,r_q_n,analysis):
# cat = ['male_c', 'breed_c', 'shunt_ty', 'portal_v', 'system_v', 'contrb_v', 'hospital', 'country', 'pps_c']
cat1 = ['breed_n', 'shunt_ty', 'portal_v', 'system_v', 'contrb_v', 'hospital', 'country', 'pps_n']
cat2 = ['shunt_ty', 'portal_v', 'system_v', 'contrb_v', 'hospital', 'country', 'pps_n']
cat3 = ['portal_v', 'system_v', 'contrb_v', 'hospital', 'country', 'pps_n']
cat4 = ['system_v', 'contrb_v', 'hospital', 'country', 'pps_n']
cat5 = ['contrb_v', 'hospital', 'country', 'pps_n']
cat6 = ['hospital', 'country', 'pps_n']
cat7 = ['country', 'pps_n']

cross_cat = pd.DataFrame()

# del(tmp)
for i in cat1:
    cross_tmp = chi_contingency(df,"male_n",i,"male_n","chi_cont")
    cross_cat = cross_cat.append(cross_tmp)

del(cross_tmp)
for i in cat2:
    cross_tmp = chi_contingency(df,"breed_n",i,"breed_n","chi_cont")
    cross_cat = cross_cat.append(cross_tmp)

del(cross_tmp)
for i in cat3:
    cross_tmp = chi_contingency(df,"shunt_ty",i,"shunt_ty","chi_cont")
    cross_cat = cross_cat.append(cross_tmp)

del(cross_tmp)
for i in cat4:
    cross_tmp = chi_contingency(df,"portal_v",i,"portal_v","chi_cont")
    cross_cat = cross_cat.append(cross_tmp)

del(cross_tmp)
for i in cat5:
    cross_tmp = chi_contingency(df,"system_v",i,"system_v","chi_cont")
    cross_cat = cross_cat.append(cross_tmp)

del(cross_tmp)
for i in cat6:
    cross_tmp = chi_contingency(df,"contrb_v",i,"contrb_v","chi_cont")
    cross_cat = cross_cat.append(cross_tmp)

del(cross_tmp)
for i in cat7:
    cross_tmp = chi_contingency(df,"hospital",i,"hospital","chi_cont")
    cross_cat = cross_cat.append(cross_tmp)

cross_tmp = chi_contingency(df,"country","pps_n","country","chi_cont")
cross_cat = cross_cat.append(cross_tmp)


# print(type(cross_cat))
# print(cross_cat)

cross_cat.head()


,r_q_n,analysis,research_q,Table,Frequency,Percent,Missing,iv1,iv2
0,male_n,chi_cont,male_n,Table male_n * breed_n,2,0.187441,.,0,1.0
1,male_n,chi_cont,male_n,Table male_n * breed_n,0,0.000000,.,0,2.0
2,male_n,chi_cont,male_n,Table male_n * breed_n,0,0.000000,.,0,3.0
3,male_n,chi_cont,male_n,Table male_n * breed_n,2,0.187441,.,0,4.0
4,male_n,chi_cont,male_n,Table male_n * breed_n,4,0.374883,.,0,5.0


# Descriptive Means first Overall then by Class

In [7]:

# df = df.astype({'pps_n': 'int64', 'contrb_v': 'int64'})
# cat_c = ['male_c', 'breed_nc', 'shunt_ty', 'portal_v', 'system_v', 'contrb_v', 'hospital', 'country', 'pps_n']
# pps_n          1078 non-null   float64

m_desc = pd.DataFrame()
m_class = pd.DataFrame()

for i in cont:
        desc_tmp = means_desc(df, i)
        m_desc = m_desc.append(desc_tmp)

print(m_desc)

for i in cat:
    for j in cont:
        desc_class_tmp = means_desc_class(df,j, i)
        m_class = m_class.append(desc_class_tmp)

print(m_class)



      ds  nmiss       n         mean  median          std     min      max  \
0  age_d      0  1082.0  1426.200739  722.50  3669.004669 -548.00  41319.0   
0  wt_kg     15  1067.0     4.828744    4.44     2.627325    0.83     12.0   
0  pps_n      4  1078.0     3.336735    4.00     1.630935    1.00      5.0   

      p25      p75  
0  298.00  1682.50  
0    2.61     6.35  
0    2.00     5.00  
        ds  level    var  nmiss      n         mean  median          std  \
0   male_n    1.0  age_d    0.0  507.0  1353.570020   693.0  3126.360063   
1   male_n    0.0  age_d    0.0  575.0  1490.242087   733.0  4089.853875   
0   male_n    1.0  wt_kg    4.0  503.0     5.216183     4.8     2.707534   
1   male_n    0.0  wt_kg   11.0  564.0     4.483209     4.0     2.505961   
0   male_n    1.0  pps_n    2.0  505.0     3.443564     4.0     1.602727   
..     ...    ...    ...    ...    ...          ...     ...          ...   
1    pps_n    5.0  pps_n    0.0  413.0     5.000000     5.0     0.00000

# Overall F Test

In [8]:

#Init Dict for DF to fill
data_dict = {}
mt_data  = pd.DataFrame()


# Set up affectional naming for stratified LOB
# lob_dict_in = {1:['lob_auto'], 2:['lob_hl'], 3:['lob_wfvc']}
lob_dict_in = {1:['df']}
lob_df = pd.DataFrame(lob_dict_in)
lob_df = lob_df.T.reset_index()
lob_df.rename(columns={0: 'ds_name', 'index': 'df_num'}, inplace = True)


# Set up affection Dict naming for DV
# dv_dict_in = {1:['sentiment'], 2:['count of calls']}
dv_dict_in = {1:['wt_kg'], 2:['age_d'], 3:['pps_n']}
dv_df = pd.DataFrame(dv_dict_in)
dv_df = dv_df.T.reset_index()
dv_df.rename(columns={0: 'dv_name', 'index':'dv_num'}, inplace=True)


# Set up Dict for Model ID to b merge from above dict
# lob_ds = {1:lob_auto, 2:lob_hl, 3:lob_wfvc}
lob_ds = {1:df}
# Set up dict for DVs
dvs = {1:'wt_kg', 2:'age_d', 3:'pps_n'}

for i in cat:
    for k1,v1 in dvs.items():
        for k,v in lob_ds.items():
            model = ols(v1 + ' ~ C(' + i + ')', data =v).fit()
            tmp = sm.stats.anova_lm(model, type=2) #typ = 2 also works
            tmp['df_num'] = k
            tmp['dv_num'] = k1
            tmp['iv'] = i
            data_dict[k] = tmp
            mt_data = mt_data.append(data_dict[k])


df_ftest = pd.merge(mt_data, lob_df, on=["df_num"])
df_ftest = pd.merge(df_ftest, dv_df, on=["dv_num"])
df_ftest



,df,sum_sq,mean_sq,F,PR(>F),df_num,dv_num,iv,ds_name,dv_name
0,1.0,1.428432e+02,1.428432e+02,2.108325e+01,4.921757e-06,1,1,male_n,df,wt_kg
1,1065.0,7.215583e+03,6.775195e+00,NaN,NaN,1,1,male_n,df,wt_kg
2,62.0,3.541848e+03,5.712658e+01,1.495010e+01,2.129818e-103,1,1,breed_n,df,wt_kg
3,990.0,3.782940e+03,3.821152e+00,NaN,NaN,1,1,breed_n,df,wt_kg
4,43.0,5.142936e+02,1.196032e+01,1.787722e+00,1.563244e-03,1,1,shunt_ty,df,wt_kg
5,1023.0,6.844132e+03,6.690256e+00,NaN,NaN,1,1,shunt_ty,df,wt_kg
6,8.0,1.937846e+02,2.422308e+01,3.577013e+00,4.177852e-04,1,1,portal_v,df,wt_kg
7,1058.0,7.164641e+03,6.771873e+00,NaN,NaN,1,1,portal_v,df,wt_kg
8,8.0,1.840302e+02,2.300377e+01,3.392340e+00,7.438002e-04,1,1,system_v,df,wt_kg
9,1058.0,7.174396e+03,6.781092e+00,NaN,NaN,1,1,system_v,df,wt_kg


# Diffs with Tukeys Adjustment

In [9]:

res = stat()
data_dict = {}
mt_data = pd.DataFrame()

for i in cat:
    for k1,v1 in dvs.items():
        for k, v in lob_ds.items():
            res.tukey_hsd(df=v, res_var=v1, xfac_var=i, anova_model=v1 + ' ~ C(' + i + ')')
            tmp = res.tukey_summary
            tmp['df_num'] = k
            tmp['dv_num'] = k1
            tmp['iv'] = i
            data_dict[k] = tmp
            mt_data = mt_data.append(data_dict[k])


dt_diffs = pd.merge(mt_data, lob_df, on=["df_num"])
dt_diffs = pd.merge(dt_diffs, dv_df, on=["dv_num"])
dt_diffs





,group1,group2,Diff,Lower,Upper,q-value,p-value,df_num,dv_num,iv,ds_name,dv_name
0,1.0,0.0,0.712831,0.182803,1.242858,3.740684e+00,0.008533,1,1,male_n,df,wt_kg
1,27.0,53.0,0.047336,-2.270016,2.364688,1.119378e-01,0.900000,1,1,breed_n,df,wt_kg
2,27.0,35.0,2.359813,0.171177,4.548449,5.908565e+00,0.017185,1,1,breed_n,df,wt_kg
3,27.0,62.0,0.626786,-3.560321,4.813893,8.203196e-01,0.900000,1,1,breed_n,df,wt_kg
4,27.0,22.0,0.910714,-4.672095,6.493524,8.939380e-01,0.900000,1,1,breed_n,df,wt_kg
...,...,...,...,...,...,...,...,...,...,...,...,...
3745,5.0,3.0,2.000000,2.000000,2.000000,4.568972e+15,0.001000,1,3,pps_n,df,pps_n
3746,5.0,4.0,1.000000,1.000000,1.000000,2.518275e+15,0.001000,1,3,pps_n,df,pps_n
3747,2.0,3.0,1.000000,1.000000,1.000000,1.821619e+15,0.001000,1,3,pps_n,df,pps_n
3748,2.0,4.0,2.000000,2.000000,2.000000,3.867426e+15,0.001000,1,3,pps_n,df,pps_n


# ANOVA of continuous vars with top 5 Breed IV

In [12]:

res = stat()
data_dict = {}
t5_breed_data = pd.DataFrame()

dvs = {1:'wt_kg', 2:'age_d', 3:'pps_n'}

cat_breed = ['breed_n']
ds_breed = {1:breed_t5}

for i in cat_breed:
    for k1,v1 in dvs.items():
        for k, v in ds_breed.items():
            res.tukey_hsd(df=v, res_var=v1, xfac_var=i, anova_model=v1 + ' ~ C(' + i + ')')
            tmp = res.tukey_summary
            tmp['df_num'] = k
            tmp['dv_num'] = k1
            tmp['iv'] = i
            data_dict[k] = tmp
            t5_breed_data = t5_breed_data.append(data_dict[k])


breed_diffs = pd.merge(t5_breed_data, lob_df, on=["df_num"])
breed_diffs = pd.merge(breed_diffs, dv_df, on=["dv_num"])
breed_diffs

,group1,group2,Diff,Lower,Upper,q-value,p-value,df_num,dv_num,iv,ds_name,dv_name
0,53.0,35.0,2.312477,1.212822,3.412132,8.175540,0.001000,1,1,breed_n,df,wt_kg
1,53.0,61.0,2.537859,1.482220,3.593498,9.346470,0.001000,1,1,breed_n,df,wt_kg
2,53.0,38.0,0.102148,-1.116693,1.320989,0.325820,0.900000,1,1,breed_n,df,wt_kg
3,53.0,47.0,1.713050,0.391189,3.034912,5.038249,0.004029,1,1,breed_n,df,wt_kg
4,35.0,61.0,0.225382,-0.679180,1.129944,0.968672,0.900000,1,1,breed_n,df,wt_kg
5,35.0,38.0,2.414625,1.324013,3.505236,8.607462,0.001000,1,1,breed_n,df,wt_kg
6,35.0,47.0,4.025527,2.820879,5.230175,12.991461,0.001000,1,1,breed_n,df,wt_kg
7,61.0,38.0,2.640007,1.593792,3.686222,9.810240,0.001000,1,1,breed_n,df,wt_kg
8,61.0,47.0,4.250909,3.086302,5.415516,14.190506,0.001000,1,1,breed_n,df,wt_kg
9,38.0,47.0,1.610902,0.296554,2.925250,4.764906,0.007774,1,1,breed_n,df,wt_kg


In [13]:
my_sheets = ['cat_one' 'cross_cat' 'm_desc' 'm_class' 'dt_diffs' 'breed_diffs']

writer = pd.ExcelWriter('ephss_out_20221219a.xlsx', engine = 'xlsxwriter')
# for i in my_sheets:
#     [i].to_excel(writer,sheet_name=i)
cat_one.to_excel(writer,sheet_name='cat_one')
cross_cat.to_excel(writer,sheet_name='cross_cat')
m_desc.to_excel(writer,sheet_name='m_desc')
m_class.to_excel(writer,sheet_name='m_class')
dt_diffs.to_excel(writer,sheet_name='dt_diffs')
breed_diffs.to_excel(writer,sheet_name='breed_diffs')
writer.save()


In [4]:
my_sheets = ['df']

writer = pd.ExcelWriter('df.xlsx', engine = 'xlsxwriter')
# for i in my_sheets:
#     [i].to_excel(writer,sheet_name=i)
df.to_excel(writer,sheet_name='data')
writer.save()


In [ ]:
# my_list = cont.copy()
# my_list += ['male_c']



# print(cont)
# print(type(cont))
# my_list = cont.copy()
# print(my_list)
# print(type(my_list))
# my_list += ['male_c']
# print(my_list)


# my_df = df[df.columns[df.columns.isin(my_list)]]

# print(my_df)
# my_df1 = df['wt_kg'].groupby(['male_c']).describe()
# df_2 = pd.DataFrame(df['wt_kg'].describe().tolist(), columns = ['Test2'])
# df_2 = pd.DataFrame(df['wt_kg'].groupby([male_c']).describe(), columns = ['Test2'])
# tmp = df.groupby(['male_c']).mean().reset_index()
# print(tmp)

# ds_counts = df["ds_size"].unique().tolist()
# dt_counts = df["month_year"].unique().tolist()

# blank_df = pd.DataFrame()
# for j in dt_counts:
#     for i in ds_counts:
#         tmp_prec = df.copy()
#         tp_prec = tmp_prec[['tot', 'recall', 'month_year']]
#         tmp_makse = (tmp_prec['tot'] == 1) & (tmp_prec['month_year'] == j)
#         tmp = tmp_prec.loc[tmp_mask]
#         tmp_df = tmp.describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.9])
#         tmp_df = tmp_df.T
#         tmp_df['ds_size'] = i
#         tmp_df['dt'] = j
#         blank_df = blank_df.append(tmp_df)

# print(blank_df.head(5))
# print(blank_df.tail(5))
# print(len(blank_df))


In [ ]:
# aov1 = aov(npl,"age_d","male_c",1)
# aov1 = aov(npl,"wt_kg","male_c",1)
# aov1 = aov(npl,"pps_n","male_c",1)
# aov1


def aovme(iv):
    # blank_aov = pd.DataFrame()
    for i in cont:
        tmp = df.copy()
        tmp_df = aov(tmp,i,iv,1)        
        tmp_df['dv'] = i
        tmp_df['iv'] = iv
        print(tmp_df)
        blank_aov = blank_aov.append(tmp_df)        
    return blank_aov
        


# cat = ['male_n', 'breed_c', 'shunt_ty', 'portal_v', 'system_v', 'contrb_v', 'hospital', 'country', 'pps_c']
# my_aov = aovme('male_c')
# my_aov = aovme('shunt_ty') # Too many levels for ANOVA
# aovme('breed') # Too many levels for ANOVA
# my_aov = aovme('portal_v') # Too many levels for ANOVA
# my_aov = aovme('system_v') # Too many levels for ANOVA
my_aov = aovme('contrb_v') # Too many levels for ANOVA


# print(blank_aov.head(5))
# print(blank_aov.tail(5))
# print(len(blank_aov))
# breed_c shunt_ty portal_v sysem_v contrb_v hospital country pps_c

: 

In [63]:
# s = sorted([str(round(i * 100, 1)) for i in npl['age_d']])
s = sorted([str(round(i * .01, 2)) for i in npl['age_d']])


s = (
    pd.Series(s)
    .groupby([str(int(float() )) for i in s])
    .apply(lambda group: "".join((x[-1] for x in group)))
    .reset_index()
)

print("\n".join(s['index'] + ' | ' + s[0]))

stemme = ("\n".join(s['index'] + ' | ' + s[0]))

stemmedf = pd.DataFrame([x.split(';') for x in stemme.split('\n')])

0 | 500000000000045688889999991111111122222222222223333333333333334444444444445555555555555555555556666666666777777777777788888888888888888889999999999222222222211111111111111222222222222222223333333444444444555555555555555666666666666667777777777788888888888888888999999993333333333333111111111222222222222222333333334444555555556666666666677777777777788888899999999994444444444444441112222222223333444444445555555555666666666777788889999955551111112223333334444555566666777788888899966661111122222233333344444555555666667777888997777777111112233333333333333444444555555666667777888999998811223335555588888889999999111222334444444455566778000111122333344566888888999991111111111111111112222233444445566789999221234555666667788899311111222223345778894442234555555666666667788899555555112223345699612222333344456667887445667777898112233333345556667788999913334455567788889001122233344588223334467799992222111222334455777788999112233567778899444235667236666788866661122333445697125667889882456899999222

In [9]:
# ds_counts = npl["ds_size"].unique().tolist()
# dt_counts = npl["month_year"].unique().tolist()
# blank_df = pd.DatFrame()
# for j in dt_counts:
#     for i in ds_counts:
#         tmp_prec = npl.copy()
#         tmp_prec = tmp_prec[['tot', 'recall', "month_year']]
#         tmp_mask = tmp_prec['tot'] == i) & (tmp_prec['monthy_year'] == j)
#         tmp = tmp_prec.loc[tmp_mask]
#         tmp_df = tmp.describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.9])
#         tmp_df = tmp_df.T
#         tmp_df['ds_size'] = i
#         tmp['dt'] = j
# 
#         blank_df = blank_df.append(tmp_df)

#         print(blank_df.head(5))
#         print(blank_df.tail(5))
#         print(len(blank_df))



blank_df = pd.DataFrame()

for i in cont:
    tmp = npl.copy()
    tmp_df = means_desc(npl,i)
    tmp_df = tmp_df.T
    tmp_df['iv'] = i
    blank_df = blank_df.append(tmp_df)

    print(blank_df.head(5))
    print(blank_df.tail(5))
    print(len(blank_df))


                  0     iv
ds            age_d  age_d
nmiss             0  age_d
n            1082.0  age_d
mean    1426.200739  age_d
median        722.5  age_d
               0     iv
std  3669.004669  age_d
min       -548.0  age_d
max      41319.0  age_d
p25        298.0  age_d
p75       1682.5  age_d
10


KeyError: "['mean', 'median', 'std', 'min', 'max', 'p25', 'p75'] not in index"

In [ ]:
import saspy, os, pandas as pd
print(saspy.__file__.replace('__init__.py', 'sascfg_personal.py'))
#  {SAS004}80898A647C57DF69657904C24EDA7E237590869733B342AC
# sas = saspy.SASsession()
sas = saspy.SASsession(cfgname='oda')
sas
hr = sas.df2sd(df)  # the short form of: hr = sas.dataframe2sasdata(hr_pd)
hr.columnInfo()
# hr.head()
# hr.means()
# hr.bar('age_d')
# hr.hist('wt_kg')
# hr.heatmap('age_d', 'male_c')
c = sas.submitLST("""
proc sgpanel data=work._df;
    PANELBY pps_n;
    hbar male_n / response=wt_kg    stat=median;
    hbar male_n / response=age_d stat=median;
run;
""")

In [ ]:
# Cntrl 1 Comment Out
# edj.rename(columns = {'SYMBOL/CUSIP':'ticker','SHARES':'vol'}, inplace = True) 
# print(type(df))
# print(df.head(2))
# df.info()
# for i in df.columns:
#     print(i)


# npl['sex'] = npl['GENDER'].astype(int)
# npl.info()
# npl = npl.astype({'wt_kgs': 'float64'})
# npl['wt'] = npl['wt_kgs'].apply(lambda x: float(x.split()[0].replace('', 'NaN')))
# npl['pps_n'].describe()
# npl = npl.astype({'wt': 'float64'})
# npl.describe()

# all_df = all_df.astype({'recall': 'float64', 'ds_size': 'float64'})
# all_df.info()

# all_df['monht_year'] = pd.to_datetime(all_df['dt']).dt.to_period("M")

# all_df['condition'] = ""

# all_df.loc[(all_df['tp'] ==1), "condtion"] = 'tp'

# all_df['gt'] = all_df.xxx.apply(lambda x: 1 if x == "Y" else 0)
# all_df = all_df[all_df.coumns[~all_df.columns.isin(['xxx'])]]


# month_data = []
# gridvars = ['tp', 'tn', 'fn', 'fp']
# for i in time_mo:
#     my_mo = df.loc[(df[month_year] == i)]
#     print(my_mo)
#     print(len(my_mo))
#     df_in = my_modfs = df_in[gridvars].sum
#     dfs = dfs.to_frame()
#     dft = dfs.transpose()
#     dft['tot'] = dft.sum(axis=1)
#     dft['tpfn'] = dft.loc[:,['tp','fn']].sum(axis=1)
#     dft['tpfp'] = dft.loc[:,['tp','fp']].sum(axis=1)
#     dft['fpfn'] = dft.loc[:,['fp','fn']].sum(axis=1)
#     dft['recall'] = dft.tp / dft.tpfn
#     dft['precision'] = dft.tp / dft.tpfn
#     dft['recall'] = dft.tp / dft.tpfp
#     dft['pct_gtc'] = dft.tpfn / dft.tot
#     dft['pct_nsc'] = dft.fpfn / dft.tot
#     dft['month_id'] = i
#     month_data.append(dft)


# month_df.rename(column={0:'tp', 1: 'tn'}, inplace = True)

# df.loc[(df['gt'] == 1) & (df['mod'] == 1), 'tp'] = 1
# df.loc[(df['gt'] == 0) & (df['mod'] == 0), 'tn'] = 1

# df['condition'] = ""
# df.loc[(df['Price'] <=100) & (df['Category']=='A'), "Flag"] = 'X'
# df.loc[(df['tp'] ==1), "condition"] = 'tp'
# df.loc[(df['tn'] ==1), "condition"] = 'tn'


# def pef_c(row):
#     if  row['date'] < '2020-03-01':
#         val = 'pre'
#     elif row['date'] >= '2020-03-01' and row['date'] < '2022-03-01':
#         val = 'during'
#     elif row['date'] >= '2022-03-01':
#         val = 'post'
#     else:
#         val = 'error'
#     return val



# def checkAge(inpAge, minAge):
#     if(inpAge>=minAge):
#         return 'senior'
#     else:
#         return 'fresher'
    
# npl['age_ty']=npl['AGE_DAYS'].apply(checkAge, args=(25,))

 
# Priting data
# print(npl.male_n)
# print(npl.male_c)

# npl['sex'] = npl['GENDER'].astype(str)


# npl['male_c'] = (dataFrame.loc[(dataFrame['Salary']>=100000) & (dataFrame['Age']< 40) & (dataFrame['JOB'].str.startswith('D')),
#                     ['Name','JOB']])
# ['sex'] = npl['GENDER'].astype(int)

# import statsmodels.api as sm
# from statsmodels.formula.api import ols
# aov1 = ols('age_d ~ C(gender)',data=df).fit()
# table = sm.stats.anova_lm(aov1, typ=2) # Type 2 Anova DataFrame
# print(table)

from saspy import SASsession
sas = SASsession(results='HTML')
print(type(sas))